## Identities point clouds analysis

In [1]:
import numpy as np
from gtda import diagrams
import matplotlib.pyplot as plt
import pickle
import plotly.graph_objects as go

MODEL_NAMES = ["retina_facenet","retina_arcface"]

/Users/mastropi/anaconda3/envs/topo/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ImportError: dlopen(/Users/mastropi/anaconda3/envs/topo/lib/python3.10/site-packages/scipy/special/_ufuncs.cpython-310-darwin.so, 0x0002): symbol not found in flat namespace '_npy_asinh'

Load the results

In [ ]:
dataset = "CelebA"
maxdim = 1

with open(f'../results/{dataset}_PH_identities_maxdim{maxdim}.pickle', 'rb') as handle:
        results = pickle.load(handle)
pers_diagrams = results["pers_diagrams"]

### Distance matrix between identity diagrams

Code to compute the distance matrix between the persistence diagrams of the indentity point clouds

In [ ]:
Dph = []
for m,model_name in enumerate(MODEL_NAMES):
  print(model_name)
  dist = diagrams.PairwiseDistance(metric = "landscape")
  Dph.append(dist.fit_transform(pers_diagrams[m]))
  

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize = (8,4))
minmin = np.min([np.min(Dph[0]), np.min(Dph[1])])
maxmax = np.max([np.max(Dph[0]), np.max(Dph[1])])
for m, model_name in enumerate(MODEL_NAMES):
    im = axes[m].imshow(Dph[m], vmin=minmin, vmax=maxmax,
                        aspect='auto',
                        cmap='viridis')
    axes[m].set_title(model_name)

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.15, 0.04, 0.7])
fig.colorbar(im, cax=cbar_ax)

### Persistence Entropy 

Code to compute a measure (like persistence entropy) of each identity point cloud's diagram 

In [ ]:
measure = []
for m,model_name in enumerate(MODEL_NAMES):
  measure.append([])
  pe = diagrams.features.PersistenceEntropy()
  measure[-1] = pe.fit_transform(pers_diagrams[m])
  measure[-1] = np.array(measure[-1])

In [ ]:
for i in range(measure[m].shape[1]):
    fig = go.Figure()
    for m, model_name in enumerate(MODEL_NAMES):
        fig.add_trace(go.Histogram(x=measure[m][:,i], name = model_name))
    fig.update_layout(barmode='overlay',width=800,height = 200, title = f"H{i}",margin=dict(l=30,r=1,b=1,t=35))
    fig.update_traces(opacity=0.5)
    fig.show()